In [13]:
#The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered 
#so that the face is more or less centered and occupies about the same amount of space in each image. 
#The task is to categorize each face based on the emotion shown in the facial expression in to one of 
#seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

#dataset can be downloaded from: 
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/overview
#Babak Rahi

# this file makes use of the "utils.py" file Also in the repository

#Babak Rahi

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from utils import getData, softmax, cost2, y2indicator, error_rate, relu
from sklearn.utils import shuffle

In [11]:
class ANN(object):
    def __init__(self, M):
        self.M = M
    
    def fit(self, X, Y, learning_rate = 10e-6, reg = 10e-1, epochs = 10000, show_fig = False):
        X, Y = shuffle(X, Y)
        Xvalid, Yvalid = X[-1000:], Y[-1000:]
        
        X, Y = X[:-1000], Y[:-1000]
        
        N, D = X.shape
        K = len(set(Y))
        T = y2indicator(Y)
        
        self.W1 = np.random.randn(D, self.M) / np.sqrt(D + self.M)
        self.b1 = np.zeros(self.M)
        self.W2 = np.random.randn(self.M, K) / np.sqrt(self.M + K)
        self.b2 = np.zeros(K)
        
        costs = []
        best_validation_error = 1
        
        for i in range(epochs):
            pY, Z = self.forward(X)
            
            self.W2 -= learning_rate*(Z.T.dot(pY - T) + reg*self.W2)
            self.b2 -= learning_rate*(np.sum(pY-T, axis=0) + reg*self.b2)
            
            # dZ = (pY-T).dot(self.W2.T) * (Z>0) #RelU
            dZ = (pY-T).dot(self.W2.T) * (1- Z * Z) #Tanh
            
            self.W1 -= learning_rate * (X.T.dot(dZ) + reg*self.W1)
            self.b1 -= learning_rate * (np.sum(dZ, axis=0) + reg*self.b1)
            
            if i % 10 == 0:
                pYvalid, _ = self.forward(Xvalid)
                c = cost2(Yvalid, pYvalid)
                costs.append(c)
                predictions = np.argmax(pYvalid, axis = 1)
                e = error_rate(Yvalid, predictions)
                
                print("i: ", i, "cost: ", c, "error: ", e)   
                if e < best_validation_error:
                    best_validation_error = e
        print("Best validation Error:", best_validation_error)
        
        if show_fig:
            plt.plot(costs)
            plt.show()
            
    def forward(self, X):
        # Z = relu(X.dot(self.W1) + self.b1)
        Z = np.tanh(X.dot(self.W1) + self.b1)
        return softmax(Z.dot(self.W2) + self.b2), Z
    
    def predict(self, X):
        pY, _ = self.forward(X)
        return np.argmax(pY, axis=1)
    
    def score(self, X, Y):
        prediction = self.predict(X)
        return 1 - error_rate(Y, prediction)
        
    

In [12]:
def main():
    X, Y, Xv, Yv = getData()
    
    model = ANN(200)
    model.fit(X, Y , show_fig = True)
    print (model.score())
    
    
if __name__ == "__main__":
    main()

i:  0 cost:  5.623618164431784 error:  0.769
i:  10 cost:  22.90162956208472 error:  0.917
i:  20 cost:  24.016515780197363 error:  0.86
i:  30 cost:  19.254036278717948 error:  0.769
i:  40 cost:  32.0142036324238 error:  0.877
i:  50 cost:  23.56062124800415 error:  0.885
i:  60 cost:  30.763590605515486 error:  0.877
i:  70 cost:  34.67636349074022 error:  0.769
i:  80 cost:  32.7961346189428 error:  0.86
i:  90 cost:  23.230256320259013 error:  0.877
i:  100 cost:  33.20977914532187 error:  0.866
i:  110 cost:  26.730315671423988 error:  0.769
i:  120 cost:  44.183496123389176 error:  0.769
i:  130 cost:  29.272836277743526 error:  0.866
i:  140 cost:  23.826071790404843 error:  0.866
i:  150 cost:  32.43555254207792 error:  0.885
i:  160 cost:  28.410113338223898 error:  0.826
i:  170 cost:  24.92190502171649 error:  0.866
i:  180 cost:  39.753503293827265 error:  0.885
i:  190 cost:  30.904893810851043 error:  0.877
i:  200 cost:  40.30345484132685 error:  0.866
i:  210 cost:  29

KeyboardInterrupt: 